In [2]:
from pymilvus import connections

# Connect to Milvus server
try:
    connections.connect("default", host="127.0.0.1", port="19530")
    print("Successfully connected to Milvus!")
    print("Active connections:", connections.list_connections())
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to Milvus!
Active connections: [('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x000002443645D370>)]


In [3]:
# Import necessary modules from pymilvus
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility
from pymilvus.exceptions import MilvusException

# Define fields for the Milvus collection, adding an "id" field as the primary key
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),  # Primary Key
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),  # Vector field
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=500)  # Text field
]

# Define the schema for the collection
schema = CollectionSchema(fields, "PDF Embeddings Collection")

# Check if the collection already exists
if utility.has_collection("pdf_collection"):
    collection = Collection("pdf_collection")
    collection.drop()  # Drop the existing collection
    print("Existing collection dropped.")

# Create the collection with the specified schema
collection = Collection("pdf_collection", schema)
print("Collection created successfully!")

# Check if an index already exists on the embedding field
index_params = {
    "metric_type": "IP",  # Use Inner Product (IP) distance for similarity search
    "index_type": "IVF_FLAT",  # Choose index type, e.g., IVF_FLAT, IVF_SQ8, HNSW, etc.
    "params": {"nlist": 128}  # nlist controls the number of clusters (higher values improve recall but increase memory usage)
}

if not collection.has_index():
    collection.create_index(field_name="embedding", index_params=index_params)
    print("Index created successfully!")
else:
    print("Index already exists!")

# Load the collection into Milvus for queries and insertions
collection.load()
print("Collection loaded successfully!")

Existing collection dropped.
Collection created successfully!
Index created successfully!
Collection loaded successfully!


In [4]:
import fitz  # PyMuPDF

# Read the PDF file
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"Error opening PDF file: {e}")
        return ""

    text = ""
    for page in doc:
        text += page.get_text()  # Extract text from each page

    return text

# Split the long text into chunks, each no longer than the specified max_length
def split_text(text, max_length=500):
    return [text[i:i + max_length] for i in range(0, len(text), max_length)]


pdf_path = "./reference/insurance/1.pdf"  # Replace with the path to your PDF file
pdf_text = extract_text_from_pdf(pdf_path)

if pdf_text:  # Check if the text was successfully extracted
    print("PDF text extracted successfully!")
    print(pdf_text[:500])  # Display only the first 500 characters to verify the output
else:
    print("No text extracted.")

PDF text extracted successfully!
第11 頁，共24 頁
銷售日期：113 年2 月26 日
延期間內發生第十六條或第十七條本公司應負保險責任之事故時，其約定之保險金計算方式將不適用，本公
司改以前項變更後之保險金額給付保險金後，本契約效力即行終止。但要保人有指定分期方式給付身故保險
金或完全失能保險金者，本公司仍依第十八條約定給付分期定額保險金。
如當時保單價值準備金扣除營業費用後的數額超過展期定期保險至滿期日所需的躉繳保險費時，要保人得以
其超過款額作為一次躉繳保險費，購買於本契約原繳費期滿時給付的繳清生存保險，其保險金額如保險單展
期定期保險生存保險金表。
要保人選擇改為展期定期保險當時，倘有增值回饋分享金、保險單借款或欠繳、墊繳保險費的情形，本公司
將以保單價值準備金加上本公司應給付的增值回饋分享金扣除欠繳保險費、借款本息、墊繳保險費本息及營
業費用後的淨額辦理。
本條營業費用以原基本保險金額之百分之一或以原基本保險金額所對應之保單價值準備金與解約金之差
額，兩者較小者為限。
第三十二條【保險單借款及契約效力的停止】
要保人繳足保險費累積達有保單價值準備金時，要保人得向本公司申請保險單借款，其可借金額上限為借款


In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the Sentence-BERT model to generate text embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Split the long text into segments
text_segments = split_text(pdf_text, max_length=500)

# Generate embeddings for each text segment and normalize them
embeddings = [model.encode(segment) for segment in text_segments]
normalized_embeddings = [embedding / np.linalg.norm(embedding) for embedding in embeddings]

# Convert normalized embeddings to list format for insertion or inspection
normalized_embeddings = [embedding.tolist() for embedding in normalized_embeddings]

# Check generated embeddings (display only the first 2 for brevity)
print(f"Generated {len(normalized_embeddings)} normalized embeddings.")
for i, embedding in enumerate(normalized_embeddings[:2]):
    print(f"Normalized Embedding {i+1}: {embedding[:10]}...")  # Show the first 10 dimensions for brevity

c:\Users\user\anaconda3\envs\rag_llm_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Generated 4 normalized embeddings.
Normalized Embedding 1: [-0.07187239080667496, 0.08410615473985672, 0.027928194031119347, -0.07848363369703293, -0.06560005992650986, -0.039626576006412506, 0.09136970341205597, 0.006937477737665176, 0.06400814652442932, 0.01294386200606823]...
Normalized Embedding 2: [-0.04443209618330002, 0.05870601162314415, 0.05654921010136604, -0.03990059345960617, -0.07484129816293716, -0.044113654643297195, 0.12110421061515808, 0.006510842591524124, 0.07214134931564331, -0.023677868768572807]...


In [6]:
# Prepare the data for insertion
# The embeddings list contains the vectors, and text_segments contains the original text
data_to_insert = [
    normalized_embeddings,       # The embeddings for each segment (vector data)
    text_segments                # The original text for each segment (text data)
]

# Insert the data into the Milvus collection
collection.insert(data_to_insert)
print("Normalized embeddings inserted into Milvus successfully!")

# Flush to make sure the data is persisted
collection.flush()

Normalized embeddings inserted into Milvus successfully!


In [7]:
# Query text and convert to embedding
query_text = "保險受益人有哪些規定?"  
query_embedding = model.encode(query_text)
normalized_query_embedding = query_embedding / np.linalg.norm(query_embedding)  # Convert query text to embedding

# Set search parameters for inner product distance
search_params = {
    "metric_type": "IP",  # Inner Product (IP) distance
    "params": {"nprobe": 50}  # nprobe controls the search accuracy, higher values yield more accurate results
}

# Perform search for the top 5 similar results
results = collection.search(
    data=[normalized_query_embedding.tolist()],
    anns_field="embedding",
    param=search_params,
    limit=5,
    output_fields=["text"]
)

# Display search results
for result in results:
    for match in result:
        print(f"ID: {match.id}, Distance: {match.distance}")
        print(f"Text:\n {match.entity.get('text')}")

ID: 453487239085688485, Distance: 0.8254454135894775
Text:
 子申
請文件）送達本公司時，本公司應即予批註或發給批註書。
身故受益人同時或先於被保險人本人身故，除要保人已另行指定受益人外，以被保險人之法定繼承人為本契
約受益人。
本條法定繼承人之順序及應得保險金之比例適用民法繼承編相關規定。
第三十六條【變更住所】
要保人的住所有變更時，應即以書面或其他約定方式通知本公司。
要保人不為前項通知者，本公司之各項通知，得以本契約所載要保人之最後住所發送之。
第三十七條【時效】
由本契約所生的權利，自得為請求之日起，經過兩年不行使而消滅。
第三十八條【批註】
本契約內容的變更，或記載事項的增刪，除第三十五條規定者外，應經要保人與本公司雙方書面或其他約定
方式同意，並由本公司即予批註或發給批註書。
第三十九條【管轄法院】
因本契約涉訟者，同意以要保人住所地地方法院為第一審管轄法院，要保人的住所在中華民國境外時，以臺
灣臺北地方法院為第一審管轄法院。但不得排除消費者保護法第四十七條及民事訴訟法第四百三十六條之九
小額訴訟管轄法院之適用。

ID: 453487239085688484, Distance: 0.8113635182380676
Text:
 保險金額，而不退還溢繳
部分的保險費。
三、因投保年齡的錯誤，而致短繳保險費者，要保人得補繳短繳的保險費或按照所付的保險費與被保險人的
真實年齡比例減少總保險金額。但在發生保險事故後始發覺且其錯誤不可歸責於本公司者，要保人不得
要求補繳短繳的保險費。
前項第一款、第二款前段情形，其錯誤原因歸責於本公司者，應加計利息退還保險費，其利息按本契約辦理
保險單借款之利率與民法第二百零三條法定週年利率兩者取其大之值計算。
第三十五條【受益人的指定及變更】
完全失能保險金的受益人，為被保險人本人，本公司不受理其指定或變更。被保險人身故時，如前述保險金
第12 頁，共24 頁
銷售日期：113 年2 月26 日
尚未給付或未完全給付，則以被保險人之法定繼承人為該部分保險金之受益人。
除前項約定外，要保人得依下列規定指定或變更受益人，並應符合指定或變更當時法令之規定：
一、於訂立本契約時，經被保險人同意指定受益人。
二、於保險事故發生前經被保險人同意變更受益人，如要保人未將前述變更

In [8]:
# Query the first 5 records in the collection, specifying fields to retrieve
results = collection.query(
    expr="id >= 0",  # Query condition, here it retrieves all records with id >= 0
    limit=10,  # Retrieve the first 5 records
    output_fields=["id", "text"]  # Specify the fields to return, such as id and text
)

# Display query results
print("Query Results:")
for result in results:
    print(f"ID: {result['id']}, Text: {result['text'][:100]}...")  # Display the first 100 characters of text

Query Results:
ID: 453487239085688482, Text: 第11 頁，共24 頁
銷售日期：113 年2 月26 日
延期間內發生第十六條或第十七條本公司應負保險責任之事故時，其約定之保險金計算方式將不適用，本公
司改以前項變更後之保險金額給付保險金後，本契...
ID: 453487239085688483, Text: 
當日保單價值準備金之百分之八十，未償還之借款本息，超過其保單價值準備金時，本契約效力即行停止。
但本公司應於效力停止日之三十日前以書面通知要保人。
本公司未依前項規定為通知時，於本公司以書面通知要保...
ID: 453487239085688484, Text: 保險金額，而不退還溢繳
部分的保險費。
三、因投保年齡的錯誤，而致短繳保險費者，要保人得補繳短繳的保險費或按照所付的保險費與被保險人的
真實年齡比例減少總保險金額。但在發生保險事故後始發覺且其錯誤不可...
ID: 453487239085688485, Text: 子申
請文件）送達本公司時，本公司應即予批註或發給批註書。
身故受益人同時或先於被保險人本人身故，除要保人已另行指定受益人外，以被保險人之法定繼承人為本契
約受益人。
本條法定繼承人之順序及應得保險金...


In [9]:
# Check the number of entities in the collection
stats = collection.num_entities
print(f"Number of entities in the collection: {stats}")

# Check the schema of the collection
schema = collection.schema
print("Collection schema:")
for field in schema.fields:
    print(f"Field name: {field.name}, Data type: {field.dtype}")

Number of entities in the collection: 4
Collection schema:
Field name: id, Data type: 5
Field name: embedding, Data type: 101
Field name: text, Data type: 21


In [10]:
import ollama

# Define a function to run the LLM model with the specified prompt
def run_ollama(prompt):
    try:
        # Run the LLM model with the specified prompt
        response = ollama.chat(
            model="llama3.1",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        # Return the generated response
        return response['message']['content']
    
    except Exception as e:
        print("Error running ollama:", e)
        return None

# Define a function to generate a response to a query using the LLM model
def generate_response(query):
    # Generate the query embedding
    results = collection.search(
        data=[normalized_query_embedding.tolist()],
        anns_field="embedding",
        param=search_params,
        limit=5,
        output_fields=["text"]
    )
    
    # Retrieve the text segments for the top 5 similar results
    retrieved_texts = "\n\n".join([match.entity.get("text") for result in results for match in result])
    prompt = f"以下是相關內容，請根據這些內容回答問題：\n\n{retrieved_texts}\n\n問題：{query}"
    
    print("Prompt length:", len(prompt))
    
    # Display the prompt to be sent to the LLM model
    print("Prompt to LLM:")
    print(prompt)
    
    # Generate a response using the LLM model
    response = run_ollama(prompt)
    return response

# Test the response generation
query = "保險受益人有哪些規定?"
response = generate_response(query)
print("LLM response:")
if response:
    print("Response generated successfully!")
    print(response)
else:
    print("No response generated.")

Prompt length: 1984
Prompt to LLM:
以下是相關內容，請根據這些內容回答問題：

子申
請文件）送達本公司時，本公司應即予批註或發給批註書。
身故受益人同時或先於被保險人本人身故，除要保人已另行指定受益人外，以被保險人之法定繼承人為本契
約受益人。
本條法定繼承人之順序及應得保險金之比例適用民法繼承編相關規定。
第三十六條【變更住所】
要保人的住所有變更時，應即以書面或其他約定方式通知本公司。
要保人不為前項通知者，本公司之各項通知，得以本契約所載要保人之最後住所發送之。
第三十七條【時效】
由本契約所生的權利，自得為請求之日起，經過兩年不行使而消滅。
第三十八條【批註】
本契約內容的變更，或記載事項的增刪，除第三十五條規定者外，應經要保人與本公司雙方書面或其他約定
方式同意，並由本公司即予批註或發給批註書。
第三十九條【管轄法院】
因本契約涉訟者，同意以要保人住所地地方法院為第一審管轄法院，要保人的住所在中華民國境外時，以臺
灣臺北地方法院為第一審管轄法院。但不得排除消費者保護法第四十七條及民事訴訟法第四百三十六條之九
小額訴訟管轄法院之適用。


保險金額，而不退還溢繳
部分的保險費。
三、因投保年齡的錯誤，而致短繳保險費者，要保人得補繳短繳的保險費或按照所付的保險費與被保險人的
真實年齡比例減少總保險金額。但在發生保險事故後始發覺且其錯誤不可歸責於本公司者，要保人不得
要求補繳短繳的保險費。
前項第一款、第二款前段情形，其錯誤原因歸責於本公司者，應加計利息退還保險費，其利息按本契約辦理
保險單借款之利率與民法第二百零三條法定週年利率兩者取其大之值計算。
第三十五條【受益人的指定及變更】
完全失能保險金的受益人，為被保險人本人，本公司不受理其指定或變更。被保險人身故時，如前述保險金
第12 頁，共24 頁
銷售日期：113 年2 月26 日
尚未給付或未完全給付，則以被保險人之法定繼承人為該部分保險金之受益人。
除前項約定外，要保人得依下列規定指定或變更受益人，並應符合指定或變更當時法令之規定：
一、於訂立本契約時，經被保險人同意指定受益人。
二、於保險事故發生前經被保險人同意變更受益人，如要保人未將前述變更通知本公司者，不得對抗本公司。
前項受益人的變更，於要保人檢具申請書及被保險人的同意書（要、被保險人為同一人時為申請書或電


In [11]:
import os
import openai


# Set up the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to call GPT-4 via OpenAI API with the specified prompt
def run_openai_gpt4(prompt):
    try:
        # Use the OpenAI API to generate a response based on the provided prompt
        response = openai.ChatCompletion.create(
            model="gpt-4",  # For GPT-4, you may also use "gpt-4-turbo" if available
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,        # Adjust max_tokens based on expected response length
            temperature=0.7        # Adjust temperature for creativity level
        )
        
        # Return the generated response content
        return response['choices'][0]['message']['content']
    
    except Exception as e:
        print("Error running GPT-4:", e)
        return None

# Function to perform the complete RAG (Retrieve-and-Generate) process
def generate_response(query):
    # Retrieval stage: Assuming we obtain the most relevant document segments
    results = collection.search(
        data=[normalized_query_embedding.tolist()],
        anns_field="embedding",
        param=search_params,
        limit=5,
        output_fields=["text"]
    )
    
    # Combine the retrieved segments into a prompt (in Chinese)
    retrieved_texts = "\n\n".join([match.entity.get("text") for result in results for match in result])
    prompt = f"以下是相關內容，請根據這些內容回答問題：\n\n{retrieved_texts}\n\n問題：{query}"
    
    print("Prompt length:", len(prompt))  # Check the prompt length to ensure it’s within limits
    
    # Print the prompt for debugging and verification
    print("Prompt to GPT-4:")
    print(prompt)
    
    # Generation stage: Call GPT-4 to generate a response based on the prompt
    response = run_openai_gpt4(prompt)
    return response

# Test query
query = "保險受益人有哪些規定？"
response = generate_response(query)
print("Generated Response:")
if response:
    print("GPT-4 Response:")
    print(response)
else:
    print("No response generated.")


Prompt length: 1984
Prompt to GPT-4:
以下是相關內容，請根據這些內容回答問題：

子申
請文件）送達本公司時，本公司應即予批註或發給批註書。
身故受益人同時或先於被保險人本人身故，除要保人已另行指定受益人外，以被保險人之法定繼承人為本契
約受益人。
本條法定繼承人之順序及應得保險金之比例適用民法繼承編相關規定。
第三十六條【變更住所】
要保人的住所有變更時，應即以書面或其他約定方式通知本公司。
要保人不為前項通知者，本公司之各項通知，得以本契約所載要保人之最後住所發送之。
第三十七條【時效】
由本契約所生的權利，自得為請求之日起，經過兩年不行使而消滅。
第三十八條【批註】
本契約內容的變更，或記載事項的增刪，除第三十五條規定者外，應經要保人與本公司雙方書面或其他約定
方式同意，並由本公司即予批註或發給批註書。
第三十九條【管轄法院】
因本契約涉訟者，同意以要保人住所地地方法院為第一審管轄法院，要保人的住所在中華民國境外時，以臺
灣臺北地方法院為第一審管轄法院。但不得排除消費者保護法第四十七條及民事訴訟法第四百三十六條之九
小額訴訟管轄法院之適用。


保險金額，而不退還溢繳
部分的保險費。
三、因投保年齡的錯誤，而致短繳保險費者，要保人得補繳短繳的保險費或按照所付的保險費與被保險人的
真實年齡比例減少總保險金額。但在發生保險事故後始發覺且其錯誤不可歸責於本公司者，要保人不得
要求補繳短繳的保險費。
前項第一款、第二款前段情形，其錯誤原因歸責於本公司者，應加計利息退還保險費，其利息按本契約辦理
保險單借款之利率與民法第二百零三條法定週年利率兩者取其大之值計算。
第三十五條【受益人的指定及變更】
完全失能保險金的受益人，為被保險人本人，本公司不受理其指定或變更。被保險人身故時，如前述保險金
第12 頁，共24 頁
銷售日期：113 年2 月26 日
尚未給付或未完全給付，則以被保險人之法定繼承人為該部分保險金之受益人。
除前項約定外，要保人得依下列規定指定或變更受益人，並應符合指定或變更當時法令之規定：
一、於訂立本契約時，經被保險人同意指定受益人。
二、於保險事故發生前經被保險人同意變更受益人，如要保人未將前述變更通知本公司者，不得對抗本公司。
前項受益人的變更，於要保人檢具申請書及被保險人的同意書（要、被保險人為同一人時為申請書或

In [12]:
import os
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API key is set correctly!")
else:
    print("API key is not set.")

API key is set correctly!
